In [46]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score, roc_curve


%matplotlib inline

In [25]:
%%time
train_df = pd.read_csv("../../01_input/train.csv")
test_df = pd.read_csv("../../01_input/test.csv")
pred_df = pd.read_csv('../../04_predict_test/oka_252-2_LightGBM_submission.csv')

CPU times: user 11.4 s, sys: 276 ms, total: 11.6 s
Wall time: 11.6 s


In [3]:
submit_flg=1 #全件処理かどうかをコントロール
grid_flg=1 #GridSearchをするかコントロール
SEED=12345
sample_num=10000
fold_num=5

#train関連
train_drop_col=['ID_code','target']
train_label='target'

#test関連
test_drop_col=['ID_code']

In [6]:
#x,y作成
x_train=train_df.drop(train_drop_col,axis=1)
y_train=train_df[train_label]
x_train_base=train_df.drop(train_drop_col,axis=1)
y_train_base=train_df[train_label]
x_test=test_df.drop(test_drop_col,axis=1)
x_train_col=x_train.columns

In [10]:
import glob
train_stack_df=pd.DataFrame(y_train)
test_stack_df=pd.DataFrame()
files = glob.glob('../../03_predict_train/*')
i=0
for file in files:
    if 'US4' in file:
        i+=1
        print(file)
        train_stack_df_tmp=pd.read_csv(file)
        train_stack_df['pred_'+str(i)]=train_stack_df_tmp

files = glob.glob('../../04_predict_test/*')
i=0
for file in files:
    if 'US4' in file:
        i+=1
        print(file)
        test_stack_df_tmp=pd.read_csv(file)
        
        if i == 1 :
            test_stack_df=test_stack_df_tmp
            test_stack_df.columns=['pred_1']
        else:
            test_stack_df['pred_'+str(i)]=test_stack_df_tmp


../../03_predict_train/US4_train_6.csv
../../03_predict_train/US4_train_0.csv
../../03_predict_train/US4_train_10.csv
../../03_predict_train/US4_train_2.csv
../../03_predict_train/US4_train_9.csv
../../03_predict_train/US4_train_12.csv
../../03_predict_train/US4_train_3.csv
../../03_predict_train/US4_train_13.csv
../../03_predict_train/US4_train_5.csv
../../03_predict_train/US4_train_14.csv
../../03_predict_train/US4_train_4.csv
../../03_predict_train/US4_train_11.csv
../../03_predict_train/US4_train_7.csv
../../03_predict_train/US4_train_8.csv
../../03_predict_train/US4_train_1.csv
../../04_predict_test/US4_test_2.csv
../../04_predict_test/US4_test_5.csv
../../04_predict_test/US4_test_7.csv
../../04_predict_test/US4_test_3.csv
../../04_predict_test/US4_test_12.csv
../../04_predict_test/US4_test_14.csv
../../04_predict_test/US4_test_1.csv
../../04_predict_test/US4_test_10.csv
../../04_predict_test/US4_test_11.csv
../../04_predict_test/US4_test_8.csv
../../04_predict_test/US4_test_4.csv

In [11]:
y_train_stack_df=train_stack_df['target']
x_train_stack_df=train_stack_df.drop('target',axis=1)
x_train_stack_df['mean']=x_train_stack_df.mean(axis=1)
x_train_stack_df['max']=x_train_stack_df.max(axis=1)
x_train_stack_df['min']=x_train_stack_df.min(axis=1)
x_train_stack_df['std']=x_train_stack_df.std(axis=1)


In [27]:
pred_df['label']=pred_df['target']

In [55]:
df_check=pd.concat([x_train_stack_df['mean'],pred_df['label']],axis=1)
df_check=pd.concat([df_check,y_train_stack_df],axis=1)
df_check=pd.concat([df_check,x_train],axis=1)

In [56]:
pd.set_option('display.max_rows', 1000)
df_check

,mean,label,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,0.036122,0.152987,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,0.696929,0.227115,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,0.023994,0.223089,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,0.491483,0.200760,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,0.258872,0.074541,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104
5,0.081136,0.049406,0,11.4763,-2.3182,12.6080,8.6264,10.9621,3.5609,4.5322,...,-6.3068,6.6025,5.2912,0.4403,14.9452,1.0314,-3.6241,9.7670,12.5809,-4.7602
6,0.293005,0.050171,0,11.8091,-0.0832,9.3494,4.2916,11.1355,-8.0198,6.1961,...,8.7830,6.4521,3.5325,0.1777,18.3314,0.5845,9.1104,9.1143,10.8869,-3.2097
7,0.199532,0.205796,0,13.5580,-7.9881,13.8776,7.5985,8.6543,0.8310,5.6890,...,13.1700,6.5491,3.9906,5.8061,23.1407,-0.3776,4.2178,9.4237,8.6624,3.4806
8,0.245156,0.049416,0,16.1071,2.4426,13.9307,5.6327,8.8014,6.1630,4.4514,...,1.4298,14.7510,1.6395,1.4181,14.8370,-1.9940,-1.0733,8.1975,19.5114,4.8453
9,0.046868,0.050942,0,12.5088,1.9743,8.8960,5.4508,13.6043,-16.2859,6.0637,...,0.5543,6.3160,1.0371,3.6885,14.8344,0.4467,14.1287,7.9133,16.2375,14.2514


In [57]:
df_check_miss=df_check[(df_check['mean']<0.3) & (df_check['target']==1)]

In [58]:
df_check_miss.describe()

,mean,label,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
count,4001.000000,4001.000000,4001.0,4001.000000,4001.000000,4001.000000,4001.000000,4001.000000,4001.000000,4001.000000,...,4001.000000,4001.000000,4001.000000,4001.000000,4001.000000,4001.000000,4001.000000,4001.000000,4001.000000,4001.000000
mean,0.171990,0.131232,1.0,10.684981,-1.539735,10.731935,6.865657,11.092267,-4.954741,5.436279,...,3.362010,7.524300,1.886514,3.167442,18.003429,-0.122089,2.359558,8.882028,15.810201,-3.179773
std,0.078192,0.146534,0.0,3.046145,4.053773,2.664171,2.038317,1.628595,7.820407,0.872242,...,4.537632,3.006784,1.450766,4.032414,3.163166,1.437002,5.467126,0.918695,3.018155,10.353799
min,0.003647,0.049406,1.0,1.665900,-14.037000,3.717300,0.961900,5.888500,-23.768800,3.007600,...,-10.196700,-0.593900,-3.317700,-9.354500,10.392000,-4.264300,-14.020400,6.363900,7.605200,-32.881200
25%,0.106484,0.054359,1.0,8.419400,-4.652500,8.688000,5.336400,9.889200,-11.053400,4.793400,...,0.030700,5.246000,0.855700,0.459300,15.644100,-1.156800,-1.838300,8.238800,13.749600,-10.863800
50%,0.176827,0.072122,1.0,10.536200,-1.472800,10.561500,6.891700,11.158300,-4.670800,5.412900,...,3.317100,7.435600,1.846000,3.223400,17.920800,-0.143500,2.517200,8.846700,15.840700,-2.634900
75%,0.240366,0.131285,1.0,12.818500,1.456600,12.572600,8.330300,12.271100,1.021000,6.025400,...,6.464900,9.616300,2.884300,6.114400,20.504200,0.857000,6.689600,9.583000,18.001300,4.882900
max,0.299951,0.919360,1.0,19.122300,8.564700,18.070500,12.630700,15.422900,15.415700,7.782100,...,16.211600,15.895800,6.872700,16.043400,27.147900,3.622600,17.161400,11.706900,25.857100,22.808700


In [59]:
train_df[train_df['target']==1].describe()

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
count,20098.0,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,...,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000,20098.000000
mean,1.0,11.156418,-1.017613,11.156633,6.864113,11.131337,-4.336522,5.581966,16.514917,0.479432,...,3.998064,7.864560,1.738266,3.120260,17.779568,-0.021130,2.688583,8.810815,15.393283,-2.532243
std,0.0,3.270293,4.220638,2.841075,2.070898,1.649266,8.140281,0.922442,3.421365,3.340028,...,4.814830,3.210779,1.482537,4.067550,3.193153,1.485975,5.556892,0.952554,3.248123,10.435910
min,1.0,0.452800,-14.037000,2.946200,0.374000,5.876200,-28.246100,2.496000,7.302400,-9.839100,...,-11.906900,-2.343000,-3.317700,-11.783400,10.120700,-5.018500,-14.020400,6.119000,6.558700,-38.852800
25%,1.0,8.695875,-4.203475,8.961125,5.314300,9.911250,-10.615800,4.882250,13.880125,-2.107425,...,0.466025,5.418550,0.669775,0.343000,15.420025,-1.103950,-1.572225,8.155250,13.232500,-10.285050
50%,1.0,11.001350,-0.992650,11.096700,6.900200,11.156300,-4.101950,5.601800,16.412700,0.579600,...,4.017500,7.846550,1.667500,3.176700,17.730500,-0.057350,2.831050,8.795400,15.427950,-1.971850
75%,1.0,13.343700,2.001375,13.047025,8.384725,12.328450,1.619950,6.218550,19.044250,3.127825,...,7.407925,10.018400,2.749050,6.063825,20.201800,0.986450,6.956250,9.517000,17.776425,5.701475
max,1.0,19.458300,9.029800,18.294100,12.706900,15.692500,16.423600,8.285200,27.039800,9.033000,...,16.746100,16.520500,7.647600,17.150400,27.295300,4.088100,17.161400,11.706900,25.857100,24.564600
